# Introduction

Python의 Model I/O에 대해 학습함

## FewShotPromptTemplate

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [10]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}')

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'question': 'What do you know about Italy?', 'answer': '\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'question': 'What do you know about Greece?', 'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Human: {question}\nAI:{answer}'), suffix='Human: What do you know about {country}?')

In [11]:

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella and Tapas
        Currency: Euro

AIMessage(content='AI: I know this:\n        Capital: Madrid\n        Language: Spanish\n        Food: Paella and Tapas\n        Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--9f5460a5-4306-4dea-9036-f04da40a9286-0')

## FewShotChatMessagePromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_66381/3954819098.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [4]:
example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}"),
  ]
)

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you kjnow about {country}?")
  ]
)

In [5]:
chain= final_prompt | chat

chain.invoke(
  {
    "country" : "Korea"
  }
)


        I know this:
        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).
        Languages: Korean
        Food: Kimchi and Bibimbap
        Currency: North Korean Won (North) and South Korean Won (South)

AIMessage(content='\n        I know this:\n        Divided into two countries: North Korea (capital: Pyongyang) and South Korea (capital: Seoul).\n        Languages: Korean\n        Food: Kimchi and Bibimbap\n        Currency: North Korean Won (North) and South Korean Won (South)', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--66ee9d8a-f93c-49a8-94b4-9374297914d5-0')

## LengthBasedExampleSelector

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_64256/3680939635.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [4]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=80,
)

In [6]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [ ]:

chain = prompt | chat

chain.invoke({"country": "Spain"})

AI: I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella and Tapas
        Currency: Euro

AIMessage(content='AI: I know this:\n        Capital: Madrid\n        Language: Spanish\n        Food: Paella and Tapas\n        Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--9f5460a5-4306-4dea-9036-f04da40a9286-0')

## Random Example Selector

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [10]:
class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples
  
  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables):
    from random import choice

    return [choice(self.examples)]

In [11]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
  examples=examples
)

In [15]:
prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector,
  suffix="Human: What do you know about {country}?",
  input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

## Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ],
)

prompt.format(country="Germmany")

'What is the capital of Germmany'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  streaming=True,
  callbacks=[
    StreamingStdOutCallbackHandler(),
  ]
)

In [17]:
intro = PromptTemplate.from_template(
  """
  You are a role playing assistant.
  And you are impersonating a {character}
  """
)

example = PromptTemplate.from_template(
  """
  this is an example of how you talk:

  Human: {example_question}
  You: {example_answer}
  """
)

start= PromptTemplate.from_template(
  """
  Strat now!

  Human: {question}
  You:
  """
)

final = PromptTemplate.from_template(
  """
  {intro}

  {example}

  {start}
  """
)

prompts = [
  ("intro", intro),
  ("example", example),
  ("start", start)
]

full_prompt = PipelinePromptTemplate(
  final_prompt=final,
  pipeline_prompts=prompts
)

chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrr, matey! Me favorite grub be a hearty plate o' salted fish and hardtack, washed down with a tankard o' grog! A pirate's feast, it be! Arg arg!

AIMessage(content="Arrrr, matey! Me favorite grub be a hearty plate o' salted fish and hardtack, washed down with a tankard o' grog! A pirate's feast, it be! Arg arg!", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--0517e49e-ee8b-4f3c-80f6-6f3f9ebf62e2-0')

## Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
  # streaming=True,
  # callbacks=[
  #   StreamingStdOutCallbackHandler(),
  # ]
)

chat.predict("How do you make italian pasta")

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_70743/3048835680.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(
/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_70743/3048835680.py:18: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict("How do you make italian pasta")


[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [llm:ChatOpenAI] [14.05s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or \"00\" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### 1. Prepare the Dough:\n- **Make a Flour Well**: On a clean surface or in a large bowl, mound the flour and make a well in the center. Add a pinch of salt.\n- **Add Eggs**: Crack the eggs into the well. Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well.\n- **Form the Dough**: As the mixture thickens, use your hands to bring it together into a dough. If the dough is too dry, add a little water, a teaspoon at a time.\n\n####

'Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or "00" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n#### 1. Prepare the Dough:\n- **Make a Flour Well**: On a clean surface or in a large bowl, mound the flour and make a well in the center. Add a pinch of salt.\n- **Add Eggs**: Crack the eggs into the well. Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well.\n- **Form the Dough**: As the mixture thickens, use your hands to bring it together into a dough. If the dough is too dry, add a little water, a teaspoon at a time.\n\n#### 2. Knead the Dough:\n- **Knead**: Once the dough comes together, knead it for about 8-10 minutes until it is smooth and elastic. If it’s sticky, sprinkle a little more flour as needed.\n- **Rest**: Wrap the dough in plastic wrap or cover it 

In [ ]:
chat.predict("How do you make italian pasta")

/var/folders/42/x0tx5h514px6gvdl09jdn4rm0000gn/T/ipykernel_71179/3587820135.py:12: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chat.predict("How do you make italian pasta")


'Making Italian pasta from scratch is a rewarding process! Here’s a simple recipe for fresh pasta:\n\n### Ingredients:\n- 2 cups all-purpose flour (or "00" flour for a more authentic texture)\n- 3 large eggs\n- A pinch of salt\n- A little water (if needed)\n\n### Instructions:\n\n1. **Prepare the Dough:**\n   - On a clean surface or in a large bowl, make a mound with the flour and create a well in the center.\n   - Add the eggs and a pinch of salt into the well.\n   - Using a fork, beat the eggs gently, gradually incorporating the flour from the edges of the well until the mixture is too stiff to mix with a fork.\n\n2. **Knead the Dough:**\n   - Once the dough starts to come together, use your hands to knead it. Knead for about 8-10 minutes until the dough is smooth and elastic. If the dough is too dry, add a little water; if it’s too sticky, sprinkle a bit more flour.\n   - Wrap the dough in plastic wrap or cover it with a cloth and let it rest for at least 30 minutes at room temperat

## Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(True)

chat = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0.1,
)

with get_openai_callback() as usage:
  a = chat.predict("What is the recipe for soju")
  b = chat.predict("What is the recipe for bread")

  # will be include usage that avobe on this line
  print(usage)
  print(usage.total_cost) # 전체 비용
  print(usage.total_tokens) # 토큰 총합
  print(usage.prompt_tokens) # 프롬프트에 사용된
  print(usage.completion_tokens) # 모델이 쓴 토큰

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the recipe for soju"
  ]
}
[llm/end] [llm:ChatOpenAI] [9.04s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Soju is a traditional Korean distilled spirit, typically made from grains or starches. The exact recipe can vary, but here’s a basic overview of how soju is traditionally made:\n\n### Ingredients:\n1. **Base Ingredient**: \n   - Traditionally, soju is made from grains like rice, barley, or wheat. Some modern versions use sweet potatoes or tapioca.\n   \n2. **Water**: \n   - Clean, filtered water is essential for the distillation process.\n\n3. **Yeast**: \n   - Yeast is used for fermentation, converting sugars into alcohol.\n\n4. **Koji (optional)**: \n   - Koji is a mold (Aspergillus oryzae) that helps in breaking down starches into sugars, which can be fermented.\n\n### Basic Steps to Make Soju:\n\n1. **Preparation of Ingredients**:\n   - If using gr

In [ ]:
# Legacy code

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#   temperature=0.1,
#   max_tokens=450,
#   model="gpt-4o-mini"
# )

# chat.save("model.json")

chat = load_llm("model.json")

chat

ValueError: Loading openai-chat LLM not supported